In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

import pandas as pd
from data_utils import load_dataset

In [ ]:
df = load_dataset("../outputs/batch_run/faith_template5/execution/Mistral-7B-Instruct-v0.2_rv16-ld0.05-bs1-p0-nta1-e3-template_engpeft-t0.2-topp0.95_executed.parquet.gzip")
df.head()

In [ ]:
print(df.iloc[0].translated_prompt)

In [ ]:
print(df.iloc[0].linked_query)

In [ ]:
print(df.iloc[0].target_raw)


In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

print(f"{tokenizer.unk_token=}")
print(f"{tokenizer.pad_token=}")
print(f"{tokenizer.unk_token_id=}")
print(f"{tokenizer.pad_token_id=}")

tokenizer.pad_token = tokenizer.unk_token

print(f"{tokenizer.unk_token=}")
print(f"{tokenizer.pad_token=}")
print(f"{tokenizer.unk_token_id=}")
print(f"{tokenizer.pad_token_id=}")

# Tokenize prompt and make files

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset
import json


In [ ]:
df = load_dataset("../outputs/batch_run/democracy_template8/execution/Mistral-7B-Instruct-v0.2_rv32-ld0.05-bs1-p0-nta1-e3-template_engpeft-t0.2-topp0.95_executed.parquet.gzip")
df.head()

In [ ]:
handpicked_data = df.loc[(~df['execution'].str.startswith("exception:")) & (~df['execution'].isnull()) & (df['execution'].map(len) > 0)].iloc[[1, 2, 3, 206, 207]]

In [ ]:
preds = tokenizer.batch_encode_plus([f"`sparql\n{i}`" for i in handpicked_data['translated_prompt'].to_list()], add_special_tokens=False, padding='longest', return_tensors='np')['input_ids'].tolist()
labels = tokenizer.batch_encode_plus([f"`sparql\n{i}`" for i in handpicked_data['target_template'].to_list()], add_special_tokens=False, padding='longest', return_tensors='np')['input_ids'].tolist()

In [ ]:
data = {
    "preds": preds,
    "labels": labels,
}

path = Path("sft_peft_compute_metrics_execute_ok.json")
path.write_text(json.dumps(data))

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset


df = load_dataset("../outputs/batch_run/lezgo/execution/Mistral-7B-Instruct-v0.2_rv16-ld0-bs1-p0-nta0-e3-template_engpeft-t0.2-topp0.95_executed.parquet.gzip")
df.head()

In [ ]:
df_out = df.head()

path = Path("evaluation_test.json")
path.write_text(df_out.to_json())

In [ ]:
data_gold = json.loads(Path("../outputs/batch_run/lezgo/preprocessed_gold.json").read_text())
df_gold = pd.read_json(data_gold['df_gold_eval'])
df_gold.head()

In [ ]:
df_gold_out = df_gold.head()

path = Path("evaluation_test_gold.json")
path.write_text(df_gold_out.to_json())

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset

import pandas as pd

In [ ]:
dataset = load_dataset("../datasets/final_queries_v1.7.json")
dataset.head()

In [ ]:
df_out = dataset[2549:2553]
df_out.head()

In [ ]:
df_out_v2 = df_out.reset_index(drop=True)
df_out_v2.head()

In [ ]:
path = Path("templatize_test.json")
path.write_text(df_out_v2.to_json())

# Sampling splits

In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset

In [14]:
df = load_dataset("../outputs/dataset_pipeline/final_fq17_with_limit_10/final_fq17-split_valid.pkl")
print(f"Size of the split: {len(df)}")

df.head()

Size of the split: 118


,basic_input,templated_input,target_raw,target_template,gold_execution,gold_executed_query
index,,,,,,
1789,"[""Find artists represented in the Swedish Nati...","[""Find artists represented in the Swedish Nati...",SELECT ?item ?itemLabel ?itemDescription ?Swed...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
2338,"[""Find all places with coordinates near Papa S...","[""Find the places located near Papa Stour isla...",SELECT ?place ?placeLabel ?location (GROUP_CON...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'place': {'type': 'uri', 'value': 'http://ww...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
2779,"[""Write a query to find the number of occurren...","[""Find the number of unique awards won by indi...",SELECT (COUNT(?item) AS ?count) ?otheraward ?o...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'otheraward': {'type': 'uri', 'value': 'http...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
1624,"[""Find all items on Wikidata that have a linke...","[""Find entities in Wikidata that have a linked...",SELECT ?item ?itemLabel ?3dmodel\nWHERE {\n?it...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
662,"[""Write a SparQL query to retrieve the name, l...","[""Write a SparQL query to retrieve the name, o...",SELECT ?doctor ?doctorLabel ?ordinal ?performe...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'doctor': {'type': 'uri', 'value': 'http://w...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...


In [15]:
entry = df.sample()
print(f"Row index: {entry.index.item()}")
for c in df.columns:
    print(c)
    data = entry[c].item()
    if isinstance(data, str):
        print(data)
    elif isinstance(data, list):
        for i in data:
            print(i)
    else:
        print("unknown type")
    print()

Row index: 1957
basic_input
"Write a SparQL query to find distinct bases named after Union soldiers and retrieve their labels in English."
"Create a SparQL query that returns the distinct bases with the P31 type of 'Military base' and P17 type of 'Civil War' and their namesake's label as a Union soldier."

templated_input
"Write a SparQL query to find military bases in the United States that are named after soldiers who served in the Union Army during the American Civil War."
"Create a SparQL query to retrieve information about military bases in the US with names derived from Union Army soldiers."
"Design a SparQL query for identifying military bases in the United States, which have been named after Union Army soldiers."

target_raw
SELECT DISTINCT ?base ?baseLabel ?namesake ?namesakeLabel WHERE {
?base wdt:P31 wd:Q245016.
?base wdt:P17 wd:Q30.
?base wdt:P138 ?namesake.
?namesake wdt:P241 wd:Q1752901.
SERVICE wikibase:label {
bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" .
}
}

# Verifying gold datasets

In [6]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset
import json
import pandas as pd

In [15]:
gold_path = Path("../outputs/dataset_pipeline/final_fq17_with_limit_10/gold_final_fq17-split_train.json")

if not gold_path.exists():
    raise FileNotFoundError("The gold dataset was not found.")

gold_data = json.loads(gold_path.read_text())
print(gold_data.keys(), "\n")

mandatory_columns = [
    "df_gold",
    "df_gold_exec_timeout",
    'df_gold_exec_fail',
    'df_gold_exec_empty',
    'df_gold_exec_to_eval',
    'df_gold_eval'
]

if len(gold_data.keys()) != len(mandatory_columns):
    raise ValueError("The gold dataset does not contain the same number of mandatory columns.")

if not all([col in mandatory_columns for col in list(gold_data.keys())]):
    raise ValueError("The gold dataset does not contain one or more of the mandatory columns.")

df_gold = pd.read_json(gold_data['df_gold']) 
df_gold_exec_timeout = pd.read_json(gold_data['df_gold_exec_timeout'])
df_gold_exec_fail = pd.read_json(gold_data['df_gold_exec_fail'])
df_gold_exec_empty = pd.read_json(gold_data['df_gold_exec_empty'])
df_gold_exec_to_eval = pd.read_json(gold_data['df_gold_exec_to_eval'])
df_gold_eval = pd.read_json(gold_data['df_gold_eval'])

print(f"{len(df_gold)=}")
print(f"{len(df_gold_exec_timeout)=}")
print(f"{len(df_gold_exec_fail)=}")
print(f"{len(df_gold_exec_empty)=}")
print(f"{len(df_gold_exec_to_eval)=}")
print(f"{len(df_gold_eval)=}", "\n")

dict_keys(['df_gold', 'df_gold_exec_timeout', 'df_gold_exec_fail', 'df_gold_exec_empty', 'df_gold_exec_to_eval', 'df_gold_eval']) 

len(df_gold)=1764
len(df_gold_exec_timeout)=0
len(df_gold_exec_fail)=0
len(df_gold_exec_empty)=0
len(df_gold_exec_to_eval)=1764
len(df_gold_eval)=1764 



In [13]:
df_gold.head()

,basic_input,templated_input,target_raw,target_template,gold_execution,gold_executed_query
1789,"[""Find artists represented in the Swedish Nati...","[""Find artists represented in the Swedish Nati...",SELECT ?item ?itemLabel ?itemDescription ?Swed...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
2338,"[""Find all places with coordinates near Papa S...","[""Find the places located near Papa Stour isla...",SELECT ?place ?placeLabel ?location (GROUP_CON...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'place': {'type': 'uri', 'value': 'http://ww...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
2779,"[""Write a query to find the number of occurren...","[""Find the number of unique awards won by indi...",SELECT (COUNT(?item) AS ?count) ?otheraward ?o...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'otheraward': {'type': 'uri', 'value': 'http...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
1624,"[""Find all items on Wikidata that have a linke...","[""Find entities in Wikidata that have a linked...",SELECT ?item ?itemLabel ?3dmodel\nWHERE {\n?it...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
662,"[""Write a SparQL query to retrieve the name, l...","[""Write a SparQL query to retrieve the name, o...",SELECT ?doctor ?doctorLabel ?ordinal ?performe...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'doctor': {'type': 'uri', 'value': 'http://w...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...


In [14]:
df_gold_eval.head()

,basic_input,templated_input,target_raw,target_template,gold_execution,gold_executed_query,gold_eval,gold_get_nested_values,gold_eval_df,gold_id_columns
1789,"[""Find artists represented in the Swedish Nati...","[""Find artists represented in the Swedish Nati...",SELECT ?item ?itemLabel ?itemDescription ?Swed...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...","[http://www.wikidata.org/entity/Q23914814, A C...",{'item': {'0': 'http://www.wikidata.org/entity...,{'item': {'0': 'http://www.wikidata.org/entity...
2338,"[""Find all places with coordinates near Papa S...","[""Find the places located near Papa Stour isla...",SELECT ?place ?placeLabel ?location (GROUP_CON...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'place': {'type': 'uri', 'value': 'http://ww...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'place': {'type': 'uri', 'value': 'http://ww...","[http://www.wikidata.org/entity/Q1552121, Papa...",{'place': {'0': 'http://www.wikidata.org/entit...,{'place': {'0': 'http://www.wikidata.org/entit...
2779,"[""Write a query to find the number of occurren...","[""Find the number of unique awards won by indi...",SELECT (COUNT(?item) AS ?count) ?otheraward ?o...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'otheraward': {'type': 'uri', 'value': 'http...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'otheraward': {'type': 'uri', 'value': 'http...","[http://www.wikidata.org/entity/Q39477935, Fel...",{'otheraward': {'0': 'http://www.wikidata.org/...,{'otheraward': {'0': 'http://www.wikidata.org/...
1624,"[""Find all items on Wikidata that have a linke...","[""Find entities in Wikidata that have a linked...",SELECT ?item ?itemLabel ?3dmodel\nWHERE {\n?it...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...","[http://www.wikidata.org/entity/Q140, http://c...",{'item': {'0': 'http://www.wikidata.org/entity...,{'item': {'0': 'http://www.wikidata.org/entity...
662,"[""Write a SparQL query to retrieve the name, l...","[""Write a SparQL query to retrieve the name, o...",SELECT ?doctor ?doctorLabel ?ordinal ?performe...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'doctor': {'type': 'uri', 'value': 'http://w...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'doctor': {'type': 'uri', 'value': 'http://w...","[http://www.wikidata.org/entity/Q33085422, Thi...",{'doctor': {'0': 'http://www.wikidata.org/enti...,{'doctor': {'0': 'http://www.wikidata.org/enti...


# Testing Collator

In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments, AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from datasets import load_dataset
from sft_peft import format_prompt
import sft_peft
from prompts_template import BASE_MISTRAL_TEMPLATE, BASE_BASIC_INSTRUCTION
from libwikidatallm.TemplateLLMQuerySender import TemplateLLMQuerySender

In [2]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.unk_token
collator = DataCollatorForCompletionOnlyLM(response_template="[/INST]", tokenizer=tokenizer)
datasets = load_dataset("pandas", data_files=
                          {
                              "valid": "../outputs/dataset_pipeline/fq17_3/fq17_3-split_valid.pkl",
                              "train": "../outputs/dataset_pipeline/fq17_3/fq17_3-split_train.pkl",
                              "test": "../outputs/dataset_pipeline/fq17_3/fq17_3-split_test.pkl",
                          })

In [4]:
datasets

DatasetDict({
    valid: Dataset({
        features: ['basic_input', 'templated_input', 'target_raw', 'target_template', 'index'],
        num_rows: 129
    })
    train: Dataset({
        features: ['basic_input', 'templated_input', 'target_raw', 'target_template', 'index'],
        num_rows: 1923
    })
    test: Dataset({
        features: ['basic_input', 'templated_input', 'target_raw', 'target_template', 'index'],
        num_rows: 513
    })
})

In [5]:
# Taken from SFTTrainer(Trainer) _prepare_non_packed_dataloader method
def _prepare_non_packed_dataloader(
    tokenizer,
    dataset,
    dataset_text_field,
    max_seq_length,
    formatting_func=None,
    add_special_tokens=True,
    remove_unused_columns=True,
):
    use_formatting_func = formatting_func is not None and dataset_text_field is None
    # self._dataset_sanity_checked = False

    # Inspired from: https://huggingface.co/learn/nlp-course/chapter7/6?fw=pt
    def tokenize(element):
        outputs = tokenizer(
            element[dataset_text_field] if not use_formatting_func else formatting_func(element),
            add_special_tokens=add_special_tokens,
            truncation=True,
            padding=False,
            max_length=max_seq_length,
            return_overflowing_tokens=False,
            return_length=False,
        )

        # if use_formatting_func and not self._dataset_sanity_checked:
        #     if not isinstance(formatting_func(element), list):
        #         raise ValueError(
        #             "The `formatting_func` should return a list of processed strings since it can lead to silent bugs."
        #         )
        #     else:
        #         self._dataset_sanity_checked = True

        return {"input_ids": outputs["input_ids"], "attention_mask": outputs["attention_mask"]}

    signature_columns = ["input_ids", "labels", "attention_mask"]

    extra_colmuns = list(set(dataset.column_names) - set(signature_columns))

    # if not remove_unused_columns and len(extra_colmuns) > 0:
    #     warnings.warn(
    #         "You passed `remove_unused_columns=False` on a non-packed dataset. This might create some issues with the default collator and yield to errors. If you want to "
    #         f"inspect dataset other columns (in this case {extra_colmuns}), you can subclass `DataCollatorForLanguageModeling` in case you used the default collator and create your own data collator in order to inspect the unused dataset columns."
    #     )

    tokenized_dataset = dataset.map(
        tokenize,
        batched=True,
        remove_columns=dataset.column_names if remove_unused_columns else None,
        num_proc=1,
        batch_size=32,
    )

    return tokenized_dataset

In [6]:
template = BASE_MISTRAL_TEMPLATE
sft_peft.templater = TemplateLLMQuerySender(None, template, start_seq='[', end_seq=']')
sft_peft.input_column = "basic_input"
sft_peft.target_column = "target_raw"

In [7]:
tokenized_dataset = _prepare_non_packed_dataloader(tokenizer, datasets['test'], None, None, format_prompt)

In [20]:
tokenized_dataset[]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 513
})

In [ ]:
collator(tokenized_dataset, return_tensors="np")

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from sft_peft import main, parse_args
import argparse


In [ ]:
args = parse_args([
    "--model", "mistralai/Mistral-7B-Instruct-v0.2",
    "--train-data", "../outputs/dataset_pipeline/fq17_3/fq17_3-split_train.pkl",
    "--target-column", "target_raw",
    "--input-column", "basic_input",
    "--valid-data", "../outputs/dataset_pipeline/fq17_3/fq17_3-split_valid.pkl",
    "--start-tag", "[query]",
    "--end-tag", "[/query]",
    "--rvalue", "32",
    "--lora-dropout", "0",
    "--batch-size", "1",
    "--gradient-accumulation", str(4),
    "--packing", "0",
    "--neft-tune-alpha", "0",
    "--epochs", "1",
    "--output", ".",
    "--save-name", "test_debug",
    "--run-name", f"test_debug",
    "--random-seed", "42"
])

In [ ]:
main(args)

In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from transformers import AutoTokenizer


In [2]:
def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids)))

In [2]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.unk_token

In [3]:
tokenizer

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-Instruct-v0.2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [12]:
print_tokens_with_ids("[/INST]")
print_tokens_with_ids("[/INST] [query]")
print_tokens_with_ids("[/INST] [query]")

[('▁[', 733), ('/', 28748), ('INST', 16289), (']', 28793)]
[('▁[', 733), ('/', 28748), ('INST', 16289), (']', 28793), ('▁[', 733), ('query', 3385), (']', 28793)]
[('▁[', 733), ('/', 28748), ('INST', 16289), (']', 28793), ('▁[', 733), ('query', 3385), (']', 28793)]


In [9]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset
from transformers import AutoTokenizer
from prompts_template import ELABORATE_INSTRUCTION, BASE_MISTRAL_TEMPLATE
from libwikidatallm.TemplateLLMQuerySender import TemplateLLMQuerySender

In [10]:
dataset = load_dataset("../datasets/final_queries_v1.7.json")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

print(ELABORATE_INSTRUCTION)

templater = TemplateLLMQuerySender(llm=None, template_text=BASE_MISTRAL_TEMPLATE, start_seq="[", end_seq="]")

dataset.head()

Your assignment involves a two-step process:
First, you will receive a task described in a single sentence. This task outlines what information you need to find or what question you need to answer.
Then, you will create a SPARQL Query: Based on the task given, you will write a SPARQL query. SPARQL is a specialized query language used to retrieve and manipulate data stored in Resource Description Framework (RDF) format. Your goal is to craft a query that, when executed, fetches the data or answers required by the initial instruction.
Make sure your SPARQL query is correctly formulated so that, upon execution, it produces the desired result matching the task's requirements.
Answer this following instruction:


,query,description,context,prompt,num_tokens,start_with_SELECT
0,SELECT ?property ?propertyType ?propertyLabel ...,Wikidata properties in numerical order,Counting stuff on Wikidata\nAll Wikidata prope...,<s>[INST] <<SYS>>This is a conversation betwee...,267,True
1,SELECT ?id ?idLabel ?idDescription ?new{\n?id ...,Wikidata properties excluding external IDs,Counting stuff on Wikidata\nVariation of the a...,<s>[INST] <<SYS>>This is a conversation betwee...,296,True
2,SELECT (COUNT(?article) AS ?count)\nWHERE {\n?...,,Counting stuff on Wikidata\nCount of scientifi...,<s>[INST] <<SYS>>This is a conversation betwee...,197,True
3,SELECT (COUNT(DISTINCT ?article) AS ?count)\nW...,Count of fictional characters,Counting stuff on Wikidata\nCount of fictional...,<s>[INST] <<SYS>>This is a conversation betwee...,203,True
4,SELECT (COUNT(?item) AS ?count)\nWHERE { ?item...,Count of items with coordinate locations,Counting stuff on Wikidata\nCount of items wit...,<s>[INST] <<SYS>>This is a conversation betwee...,186,True


In [20]:
longest_query = dataset['query'].loc[dataset['query'].map(len).argmax()]

In [23]:
def prepare_prompts(query, templater:TemplateLLMQuerySender, template:str, start_tag:str = '`sparql\n', end_tag:str = "`"):
    start_tag = '`sparql\n'
    end_tag = "`"

    x = templater.apply_template({
        "system_prompt": template,
        "prompt": "\""+ ("abigword " * 20) +"\""
    })
    x += f"{start_tag}{query}{end_tag}"
    return x

longest_prompt = prepare_prompts(longest_query, templater=templater, template=ELABORATE_INSTRUCTION, start_tag="`sparql\n", end_tag="`")
len(tokenizer(longest_prompt)["input_ids"])

2739

# Looking at generated texts during evaluation

In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

import json
import random
from prompts_template import ELABORATE_INSTRUCTION
import sft_peft
from sft_peft import is_query_format_acceptable, execute_pipeline, execute_query, extract_query
from execution_utils import is_query_empty, can_add_limit_clause, add_relevant_prefixes_to_query, send_query_to_api


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# folder_path = Path("../tests/scriptstests/data/compute_metrics_real_data_from_training_1")
folder_path = Path("../outputs/batch_run/nt_mis_lla3/models")
run_name = "M-7B-I-v0.2_optawbn8-cdfp16-lr000005-rv128-ramul2-ld005-bs2-ga8-gc1-p0-nta0-e3-ctx3072-q4bit-template-template-stsparql"
# run_name = "M-L-3-8B-I_optawbn8-cdfp16-lr000005-rv128-ramul2-ld005-bs2-ga8-gc1-p0-nta0-e3-ctx3072-q4bit-template-template-stsparql"
num_epochs = 3
epochs_data = [json.loads((folder_path / f"{run_name.replace('_adapters', '')}_compute_metrics_{i}.json").read_text()) for i in range(num_epochs)]

len(epochs_data)

3

In [3]:
columns = epochs_data[0].keys()
print(columns)

dict_keys(['preds', 'generated_texts', 'tokenized_preds', 'raw_target_queries', 'str_labels', 'str_inputs'])


In [7]:
epochs_data[0]['str_inputs']

['[INST] Your assignment involves a two-step process:\nFirst, you will receive a task described in a single sentence. This task outlines what information you need to find or what question you need to answer. Read it properly.\nThen, you will create a SPARQL Query: Based on the task given, you will write a SPARQL query. SPARQL is a specialized query language used to retrieve and manipulate data stored in Resource Description Framework (RDF) format. Your goal is to craft a query that, when executed, fetches the data or answers required by the initial instruction.\nMake sure your SPARQL query is correctly formulated so that, upon execution, it produces the desired result matching the task\'s requirements.\nAnswer this following instruction: "Create a SparQL query that returns Bengali lexemes along with their examples, senses, and the provenance of the data." [/INST] ',
 '[INST] Your assignment involves a two-step process:\nFirst, you will receive a task described in a single sentence. Thi

In [4]:
def print_label_answer_and_preds(row_number:int = 0, num_epochs:int = 3):
    print(f"LABEL n°{row_number}","=" * 98)
    print(epochs_data[0]["str_labels"][row_number])
    for i in range(num_epochs):
        print(f"EPOCH {num_epochs - i - 1}","=" * 100)
        print(epochs_data[num_epochs - i - 1]["generated_texts"][row_number])
    
def get_execution_response(query: str):
    query = extract_query(query)
    if is_query_empty(query):
        return None
    else:
        query = add_relevant_prefixes_to_query(query)
    
        if can_add_limit_clause(query):
            query += f"\nLIMIT 10"

    response = send_query_to_api(query, do_print=False)
    return response

In [5]:
print(ELABORATE_INSTRUCTION)

Your assignment involves a two-step process:
First, you will receive a task described in a single sentence. This task outlines what information you need to find or what question you need to answer. Read it properly.
Then, you will create a SPARQL Query: Based on the task given, you will write a SPARQL query. SPARQL is a specialized query language used to retrieve and manipulate data stored in Resource Description Framework (RDF) format. Your goal is to craft a query that, when executed, fetches the data or answers required by the initial instruction.
Make sure your SPARQL query is correctly formulated so that, upon execution, it produces the desired result matching the task's requirements.
Answer this following instruction:


In [10]:
n_rows = len(epochs_data[0]["generated_texts"])

row_number = random.randint(0, n_rows)
print_label_answer_and_preds(row_number, num_epochs)

LABEL n°7 ==================================================================================================

[sparql]PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>

SELECT DISTINCT ?item ?itemLabel ?pic ?linkTo ?linkToLabel # ?instance ?instanceLabel ?subClass ?subClassLabel
WHERE
{
wd:[entity:power] ^wdt:[property:subclass of]* ?item .
?item ^wdt:[property:subclass of] ?linkTo .
?linkTo wdt:[property:instance of] ?instance.
?linkTo wdt:[property:subclass of]* ?subClass.
OPTIONAL { ?item wdt:[property:image] ?pic }
MINUS {?linkTo wdt:[property:country] | wdt:[property:applies to jurisdiction] ?jurisdiction}
MINUS {?item wdt:[property:country] | wdt:[property:applies to jurisdiction] ?jurisdiction}
SERVICE wikibase:label {bd:serviceParam wikibase:language "en" }
FILTER (?instance NOT IN (wd:[entity:profession], wd:[entity:Wikimedia project page], wd:[e

In [8]:
decoded_label = epochs_data[0]["decoded_labels"][row_number]
decoded_pred = epochs_data[9]["decoded_preds"][row_number]

sft_peft.start_tag = "[sparql]"
sft_peft.end_tag = "[/sparql]"

is_query_format_acceptable_label = is_query_format_acceptable(decoded_label)
is_query_format_acceptable_pred = is_query_format_acceptable(decoded_pred)
print(f"Label is acceptable query: {is_query_format_acceptable_label}")
print(f"Pred is acceptable query: {is_query_format_acceptable_pred}")
print()

if not is_query_format_acceptable_label or not is_query_format_acceptable_pred:
    raise Exception("Decoded label or pred is not a valid SPARQL query.")

pipelined_label = execute_pipeline([decoded_label])[0]
pipelined_pred = execute_pipeline([decoded_pred])[0]

print(f"Pipelined label success: {not pipelined_label['has_error']}")
print(f"Pipelined pred success: {not pipelined_pred['has_error']}")
print()

if pipelined_label['has_error'] or pipelined_pred['has_error']:
    raise Exception('Pipelined label or pred failed')

output_label = pipelined_label.get('output', None)
output_pred = pipelined_pred.get('output', None)

# print(f"Output Label: {output_label}")
# print(f"Output Prediction: {output_pred}")

extracted_label = extract_query(output_label)
extracted_pred = extract_query(output_pred)

print(f"Successfully extracted query label={extracted_label != None}")
print(f"Successfully extracted query pred={extracted_pred != None}")
print()

if extracted_label is None or extracted_pred is None:
    raise Exception('Failed to extract query')

print("Extracted Label")
print(f"{extracted_label}")
print()
print("Extracted Prediction")
print(f"{extracted_pred}")
print()

executed_label = execute_query(output_label)
executed_pred  = execute_query(output_pred)

print("executed_label")
print(f"{executed_label if executed_label != None else 'Execution has failed!'}")
print()
print("executed_pred")
print(f"{executed_pred if executed_pred != None else 'Execution has failed!'}")
if executed_pred == None:
    response = get_execution_response(output_pred)
    print(response)

Label is acceptable query: True
Pred is acceptable query: True

Pipelined label success: True
Pipelined pred success: True

Successfully extracted query label=True
Successfully extracted query pred=True

Extracted Label
PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>

SELECT DISTINCT ?person ?personLabel ?countryLabel ?cpan ?github ?wikimedia {
?person wdt:P31 wd:Q5 .
{
{ ?person wdt:P8124 ?cpan } UNION
{ ?person wdt:P2037 ?github } UNION
{ ?person wdt:P4174 ?wikimedia }
}
{
{ ?person wdt:P106 wd:Q83307 } # occupation minister
UNION {
?person wdt:P39 ?pos . # position held: minister
?pos wdt:P279* wd:Q83307
}
}
OPTIONAL { ?person wdt:P27 ?country }
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}

Extracted Prediction
PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <ht

In [19]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, load_peft_weights
from prompts_template import ELABORATE_INSTRUCTION, BASE_MISTRAL_TEMPLATE
import torch
import os

In [3]:
model_path = "mistralai/Mistral-7B-Instruct-v0.2"
adapter_path = "../outputs/batch_run/learning_experiment_long_epochs_4/models/M-7B-I-v0.2_oawtor-rv256-rm1-ld0-bs2-ga8-gc1-p0-nta0-e10-ctx3072-qno-template-template-stsparql_adapters"
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map=device,
    torch_dtype=dtype,
)

try:
    model = PeftModel.from_pretrained(model, adapter_path, is_trainable=False)
except RuntimeError as e: # RuntimeError => Might be padding token is left in the embedding when saved
    d_adpt = load_peft_weights(adapter_path)
    len_d_adpt = len(d_adpt.get('base_model.model.lm_head.weight', 0))
    if len_d_adpt != 0 and len_d_adpt != len(tokenizer):
        model.resize_token_embeddings(len(tokenizer) + 1)
        model = PeftModel.from_pretrained(model, adapter_path, is_trainable=False)
    else:
        raise e
except Exception as e:
    try:
        model = PeftModel.from_pretrained(model, os.path.abspath(adapter_path), is_trainable=False)
    except RuntimeError as e: # RuntimeError => Might be padding token is left in the embedding when saved
        d_adpt = load_peft_weights(adapter_path)
        len_d_adpt = len(d_adpt.get('base_model.model.lm_head.weight', 0))
        if len_d_adpt != 0 and len_d_adpt != len(tokenizer):
            model.resize_token_embeddings(len(tokenizer) + 1)
            model = PeftModel.from_pretrained(model, os.path.abspath(adapter_path), is_trainable=False)
        else:
            raise e
    except Exception as e:
        raise e

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
d_adpt = load_peft_weights(adapter_path)
len(d_adpt.get('base_model.model.lm_head.weight', 0))

32001

In [25]:
list(d_adpt.keys())

['base_model.model.lm_head.weight',
 'base_model.model.model.embed_tokens.weight',
 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.weight',
 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.weight',
 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.weight',
 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.weight',
 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.weight',
 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.weight',
 'base_model.model.model.layers.1.self_attn.v_proj.lora_A.weight',
 'base_model.model.model.layers.1.self_attn.v_proj.lora_B.weight',
 'base_model.model.model.layers.10.self_attn.q_proj.lora_A.weight',
 'base_model.model.model.layers.10.self_attn.q_proj.lora_B.weight',
 'base_model.model.model.layers.10.self_attn.v_proj.lora_A.weight',
 'base_model.model.model.layers.10.self_attn.v_proj.lora_B.weight',
 'base_model.model.model.layers.11.self_attn.q_proj.lora_A.weight',
 'base_model.model.model.layers.11.self_a

In [14]:
model.resize_token_embeddings(len(tokenizer))

Embedding(32000, 4096)

In [8]:
modified_tokenizer = AutoTokenizer.from_pretrained(model_path)
modified_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [12]:
print(f"{modified_tokenizer.pad_token=}")
print(f"{modified_tokenizer.pad_token_id=}")

modified_tokenizer.pad_token='[PAD]'
modified_tokenizer.pad_token_id=32000


In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset
from transformers import AutoTokenizer
from generate_finetune_dataset import keep_working_queries
import random

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
path = Path("../outputs/dataset_pipeline/fq17_3/fq17_3-generated_prompt-executed.parquet.gzip")
if not path.exists():
    raise FileExistsError(f"No file was found at this path: {path}")

dataset = load_dataset(path)
dataset.head()

,query,description,context,prompt,num_tokens,start_with_SELECT,query_templated,basic_prompt,basic_num_tokens,basic_result,...,basic_is_skipped,basic_is_prompt_too_long,templated_prompt,templated_num_tokens,templated_result,templated_full_answer,templated_is_skipped,templated_is_prompt_too_long,execution,executed_query
index,,,,,,,,,,,,,,,,,,,,,
0,SELECT ?property ?propertyType ?propertyLabel ...,Wikidata properties in numerical order,Counting stuff on Wikidata\nAll Wikidata prope...,<s>[INST] <<SYS>>This is a conversation betwee...,267,True,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"<s>[INST] QUERY=""SELECT ?property ?propertyTyp...",226,"[""Write a SparQL query to retrieve Wikidata pr...",...,False,False,"<s>[INST] QUERY=""PREFIX bd: <http://www.bigdat...",289,"[""Write a SparQL query to retrieve all Wikidat...","{'content': ' 1. ""Write a SparQL query to retr...",False,False,"[{'property': {'type': 'uri', 'value': 'http:/...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
1,SELECT ?id ?idLabel ?idDescription ?new{\n?id ...,Wikidata properties excluding external IDs,Counting stuff on Wikidata\nVariation of the a...,<s>[INST] <<SYS>>This is a conversation betwee...,296,True,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"<s>[INST] QUERY=""SELECT ?id ?idLabel ?idDescri...",259,"[""Write a SparQL query to retrieve the ID, lab...",...,False,False,"<s>[INST] QUERY=""PREFIX bd: <http://www.bigdat...",322,"[""Write a SparQL query to retrieve the ID, lab...","{'content': ' 1. ""Write a SparQL query to retr...",False,False,"[{'id': {'type': 'uri', 'value': 'http://www.w...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
2,SELECT (COUNT(?article) AS ?count)\nWHERE {\n?...,,Counting stuff on Wikidata\nCount of scientifi...,<s>[INST] <<SYS>>This is a conversation betwee...,197,True,PREFIX wd: <http://www.wikidata.org/entity/>\n...,"<s>[INST] QUERY=""SELECT (COUNT(?article) AS ?c...",159,"[""Find the total number of scientific articles...",...,False,False,"<s>[INST] QUERY=""PREFIX wd: <http://www.wikida...",201,"[""Write a SparQL query to retrieve the count o...","{'content': ' 1. ""Write a SparQL query to retr...",False,False,timeout,PREFIX wd: <http://www.wikidata.org/entity/>\n...
3,SELECT (COUNT(DISTINCT ?article) AS ?count)\nW...,Count of fictional characters,Counting stuff on Wikidata\nCount of fictional...,<s>[INST] <<SYS>>This is a conversation betwee...,203,True,PREFIX wd: <http://www.wikidata.org/entity/>\n...,"<s>[INST] QUERY=""SELECT (COUNT(DISTINCT ?artic...",165,"[""Find the number of distinct fictional charac...",...,False,False,"<s>[INST] QUERY=""PREFIX wd: <http://www.wikida...",210,"[""Write a query to find the total number of di...","{'content': ' 1. ""Write a query to find the to...",False,False,[{'count': {'datatype': 'http://www.w3.org/200...,PREFIX wd: <http://www.wikidata.org/entity/>\n...
4,SELECT (COUNT(?item) AS ?count)\nWHERE { ?item...,Count of items with coordinate locations,Counting stuff on Wikidata\nCount of items wit...,<s>[INST] <<SYS>>This is a conversation betwee...,186,True,PREFIX wdt: <http://www.wikidata.org/prop/dire...,"<s>[INST] QUERY=""SELECT (COUNT(?item) AS ?coun...",148,"[""How many items in Wikidata have recorded coo...",...,False,False,"<s>[INST] QUERY=""PREFIX wdt: <http://www.wikid...",172,"[""Write a SparQL query to retrieve the number ...","{'content': ' 1. ""Write a SparQL query to retr...",False,False,[{'count': {'datatype': 'http://www.w3.org/200...,PREFIX wdt: <http://www.wikidata.org/prop/dire...


In [3]:
dataset.keys()

Index(['query', 'description', 'context', 'prompt', 'num_tokens',
       'start_with_SELECT', 'query_templated', 'basic_prompt',
       'basic_num_tokens', 'basic_result', 'basic_full_answer',
       'basic_is_skipped', 'basic_is_prompt_too_long', 'templated_prompt',
       'templated_num_tokens', 'templated_result', 'templated_full_answer',
       'templated_is_skipped', 'templated_is_prompt_too_long', 'execution',
       'executed_query'],
      dtype='object')

In [4]:
dropped_dataset = keep_working_queries(dataset)

columns_to_take = [
    "templated_result",
    "query_templated",
]

dataset_for_tokenization = dropped_dataset[columns_to_take]
dataset_for_tokenization.head()

,templated_result,query_templated
index,,
0,"[""Write a SparQL query to retrieve all Wikidat...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
1,"[""Write a SparQL query to retrieve the ID, lab...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
3,"[""Write a query to find the total number of di...",PREFIX wd: <http://www.wikidata.org/entity/>\n...
4,"[""Write a SparQL query to retrieve the number ...",PREFIX wdt: <http://www.wikidata.org/prop/dire...
5,"[""Write a SparQL query to retrieve the number ...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...


In [5]:
all_concat_dataset = dataset_for_tokenization.apply(lambda x: x['templated_result'][0].strip() + x["query_templated"].strip(), axis=1).sum()
print(f"{len(all_concat_dataset)=}")

len(all_concat_dataset)=2294959


In [6]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-Instruct-v0.2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
tokens = tokenizer(all_concat_dataset)
type(tokens)

transformers.tokenization_utils_base.BatchEncoding

In [8]:
print(f"{len(tokens.tokens())=}")

len(tokens.tokens())=824341


In [27]:
entry_iloc_id = random.randint(0, len(dropped_dataset) - 1)
entry = dropped_dataset.iloc[entry_iloc_id]
print(f"Entry n°{entry_iloc_id}")
print("----- Prompts From Original -----")
print("\n".join(entry["basic_result"]))
print("----- Original Query -----")
print(entry["query"], "\n")
print("----- Prompts From Templated -----")
print("\n".join(entry["templated_result"]))
print("----- Templated Query -----")
print(entry["query_templated"])

Entry n°353
----- Prompts From Original -----
"Write a SPARQL query to retrieve all books with a title, illustrator, publisher, and publication date, but allow for the possibility that some books may not have an illustrator or publisher listed."
"Create a SPARQL query to find all books with a title, optional illustrator and publisher information, and a publication date."
"Generate a SPARQL query to retrieve books with required title, publication date, and optional illustrator and publisher information."
----- Original Query -----
SELECT ?book ?title ?illustratorLabel ?publisherLabel ?published
WHERE
{
?book wdt:P50 wd:Q35610. # required wdt:P50
OPTIONAL { # match all or none from group:
?book wdt:P1476 ?title; # required wdt:P1476
wdt:P110 ?illustrator; # required wdt:P110
wdt:P123 ?publisher; # required wdt:P123
wdt:P577 ?published. # required wdt:P577
}
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
} 

----- Prompts From Templated -----
"Write a S

In [3]:
dataset.keys()

Index(['query', 'description', 'context', 'prompt', 'num_tokens',
       'start_with_SELECT', 'query_templated', 'basic_prompt',
       'basic_num_tokens', 'basic_result', 'basic_full_answer',
       'basic_is_skipped', 'basic_is_prompt_too_long', 'templated_prompt',
       'templated_num_tokens', 'templated_result', 'templated_full_answer',
       'templated_is_skipped', 'templated_is_prompt_too_long', 'execution',
       'executed_query'],
      dtype='object')

In [9]:
print(dataset['templated_prompt'].iloc[0])
print(dataset['templated_result'].iloc[0])

<s>[INST] QUERY="PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX wikibase: <http://wikiba.se/ontology#>

SELECT ?property ?propertyType ?propertyLabel ?propertyDescription WHERE {
?property wikibase:propertyType ?propertyType .
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
} ORDER BY ASC(xsd:integer(STRAFTER(STR(?property), 'P')))" DESCRIPTION="Wikidata properties in numerical order" CONTEXT="Counting stuff on Wikidata
All Wikidata properties with label and description, ordered numerically
Adapted from one of the Query Service Examples" Read QUERY, DESCRIPTION, and CONTEXT. There is a machine capable of writing the given SparQL QUERY if we provide the correct prompt. A prompt should be at least one sentence and no longer than five sentences. Each prompt must be enclosed in quotation marks. Provide a list of three prompts that would elicit the QUERY. [/INST]
['"Write a SparQL query to retrieve all Wikid

In [5]:
QUERY="PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX wikibase: <http://wikiba.se/ontology#>

SELECT ?property ?propertyType ?propertyLabel ?propertyDescription WHERE {
?property wikibase:propertyType ?propertyType .
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
} ORDER BY ASC(xsd:integer(STRAFTER(STR(?property), 'P')))" DESCRIPTION="Wikidata properties in numerical order" CONTEXT="Counting stuff on Wikidata
All Wikidata properties with label and description, ordered numerically
Adapted from one of the Query Service Examples"

query
description
context
prompt
num_tokens
start_with_SELECT
query_templated
basic_prompt
basic_num_tokens
basic_result
basic_full_answer
basic_is_skipped
basic_is_prompt_too_long
templated_prompt
templated_num_tokens
templated_result
templated_full_answer
templated_is_skipped
templated_is_prompt_too_long
execution
executed_query


In [2]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from transformers import AutoTokenizer

In [3]:
prompt = """<s> [INST] Your assignment involves a two-step process:                                                                                    
First, you will receive a task described in a single sentence. This task outlines what information you need to find or what question you need to answer.                        
Then, you will create a SPARQL Query: Based on the task given, you will write a SPARQL query. SPARQL is a specialized query language used to retrieve and manipulate data stored
 in Resource Description Framework (RDF) format. Your goal is to craft a query that, when executed, fetches the data or answers required by the initial instruction.
Make sure your SPARQL query is correctly formulated so that, upon execution, it produces the desired result matching the task's requirements.
Answer this following instruction:                                                      
"Write a SparQL query to find the top 10 properties that connect warriors or military personnel to military organizations in Wikidata, ordered by frequency of usage."
[/INST] [sparql]PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX hint: <http://www.bigdata.com/queryHints#>
PREFIX wd: <http://www.wikidata.org/entity/>                                                                                                                                    
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>                                                                                                                                   
                                            
SELECT ?count ?property ?propertyLabel ?samp WHERE {                                                                                                                            
{                                                                                       
SELECT (COUNT(?item) AS ?count) ?property (SAMPLE(?item) AS ?samp) WHERE {                                                                                                      
hint:Query hint:optimizer "None".  
?org_class wdt:[property:subclass of]* wd:[entity:armed organization].                                                                                                          
?org wdt:[property:instance of]? ?org_class .
?item ?prop ?org .                                                                      
?item wdt:[property:occupation] ?role .
?role wdt:[property:subclass of]* wd:[entity:warrior] .                                                                                                                         
?property wikibase:directClaim ?prop .
} GROUP BY ?property                                                                                                                                                            
}                                                                                                                                                                               
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}                                 
ORDER BY DESC (?count)[/sparql]"""

In [4]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/CodeLlama-7b-Instruct-hf")

In [6]:
tokenizer("[/INST]")

{'input_ids': [1, 518, 29914, 25580, 29962], 'attention_mask': [1, 1, 1, 1, 1]}

In [5]:
tokenizer(prompt)

{'input_ids': [1, 1, 29871, 518, 25580, 29962, 3575, 12827, 20789, 263, 1023, 29899, 10568, 1889, 29901, 462, 462, 462, 462, 462, 268, 13, 6730, 29892, 366, 674, 7150, 263, 3414, 5439, 297, 263, 2323, 10541, 29889, 910, 3414, 714, 9012, 825, 2472, 366, 817, 304, 1284, 470, 825, 1139, 366, 817, 304, 1234, 29889, 462, 308, 13, 11760, 29892, 366, 674, 1653, 263, 10937, 1718, 2239, 13641, 29901, 16564, 373, 278, 3414, 2183, 29892, 366, 674, 2436, 263, 10937, 1718, 2239, 2346, 29889, 10937, 1718, 2239, 338, 263, 4266, 1891, 2346, 4086, 1304, 304, 10563, 322, 26749, 848, 6087, 13, 297, 18981, 12953, 16657, 313, 29934, 4037, 29897, 3402, 29889, 3575, 7306, 338, 304, 25554, 263, 2346, 393, 29892, 746, 8283, 29892, 6699, 267, 278, 848, 470, 6089, 3734, 491, 278, 2847, 15278, 29889, 13, 9984, 1854, 596, 10937, 1718, 2239, 2346, 338, 5149, 883, 7964, 577, 393, 29892, 2501, 8225, 29892, 372, 13880, 278, 7429, 1121, 9686, 278, 3414, 29915, 29879, 11780, 29889, 13, 22550, 445, 1494, 15278, 29901, 46

In [9]:
input_ids = tokenizer(prompt)["input_ids"]
list(map(lambda x: print(f"{x} => {tokenizer.decode(x)}"), input_ids))

1 => <s>
1 => <s>
29871 => 
518 => [
25580 => INST
29962 => ]
3575 => Your
12827 => assignment
20789 => involves
263 => a
1023 => two
29899 => -
10568 => step
1889 => process
29901 => :
462 =>                
462 =>                
462 =>                
462 =>                
462 =>                
268 =>    
13 => 

6730 => First
29892 => ,
366 => you
674 => will
7150 => receive
263 => a
3414 => task
5439 => described
297 => in
263 => a
2323 => single
10541 => sentence
29889 => .
910 => This
3414 => task
714 => out
9012 => lines
825 => what
2472 => information
366 => you
817 => need
304 => to
1284 => find
470 => or
825 => what
1139 => question
366 => you
817 => need
304 => to
1234 => answer
29889 => .
462 =>                
308 =>        
13 => 

11760 => Then
29892 => ,
366 => you
674 => will
1653 => create
263 => a
10937 => SP
1718 => AR
2239 => QL
13641 => Query
29901 => :
16564 => Based
373 => on
278 => the
3414 => task
2183 => given
29892 => ,
366 => you
674 => will
2436 => writ

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
[29961, 29914, 25580, 29962]

In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset

In [2]:
path = "../datasets/final_queries_v1.7.json"
dataset = load_dataset(path)

dataset.head()

,query,description,context,prompt,num_tokens,start_with_SELECT
0,SELECT ?property ?propertyType ?propertyLabel ...,Wikidata properties in numerical order,Counting stuff on Wikidata\nAll Wikidata prope...,<s>[INST] <<SYS>>This is a conversation betwee...,267,True
1,SELECT ?id ?idLabel ?idDescription ?new{\n?id ...,Wikidata properties excluding external IDs,Counting stuff on Wikidata\nVariation of the a...,<s>[INST] <<SYS>>This is a conversation betwee...,296,True
2,SELECT (COUNT(?article) AS ?count)\nWHERE {\n?...,,Counting stuff on Wikidata\nCount of scientifi...,<s>[INST] <<SYS>>This is a conversation betwee...,197,True
3,SELECT (COUNT(DISTINCT ?article) AS ?count)\nW...,Count of fictional characters,Counting stuff on Wikidata\nCount of fictional...,<s>[INST] <<SYS>>This is a conversation betwee...,203,True
4,SELECT (COUNT(?item) AS ?count)\nWHERE { ?item...,Count of items with coordinate locations,Counting stuff on Wikidata\nCount of items wit...,<s>[INST] <<SYS>>This is a conversation betwee...,186,True


In [19]:
# assuming 'df' is your DataFrame and it has these columns
row = dataset[['query', 'description', 'context']].loc[(dataset['description'].str.len() > 0)&(dataset['context'].str.len() > 0)].sample()
print("Query")
print(row['query'].item())
print("Description")
print(row['description'].item())
print("Context")
print(row['context'].item())


Query
SELECT ?jobname (COUNT(?person) AS ?members)
WHERE {
?person wdt:P3429 ?ee;
wdt:P106 ?job.
?job rdfs:label ?jobname filter (lang(?jobname) = "en")
} GROUP BY ?jobname
ORDER BY DESC (?members)
Description
BubbleChart
Context
Biographical facts
Bubble chart of occupations


In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset

In [4]:
dataset = load_dataset("../outputs/batch_run/test/execution/CL-7b-Instruct_oawbn8-rv256-rm1-ld0-bs2-ga8-gc1-p0-nta0-e1-ctx3072-qno-template-template-stsparql_engpeft-dgreedy-t02-topp095-cdbf16_executed.parquet.gzip")
dataset.head()

,row,last_executed_step,to_be_executed_step,translated_prompt,status,has_error,output,basic_input,templated_input,target_raw,target_template,gold_execution,gold_executed_query,execution,executed_query
index,,,,,,,,,,,,,,,
1789,"""Find artists represented in the Swedish Natio...",,LLMTranslator,,"Unexpected err.msg=""The LLM result doesn't mat...",True,None,"[""Find artists represented in the Swedish Nati...","""Find artists represented in the Swedish Natio...",SELECT ?item ?itemLabel ?itemDescription ?Swed...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,exception: query is empty,None
2338,"""Find the places located near Papa Stour islan...",,LLMTranslator,,"Unexpected err.msg=""The LLM result doesn't mat...",True,None,"[""Find all places with coordinates near Papa S...","""Find the places located near Papa Stour islan...",SELECT ?place ?placeLabel ?location (GROUP_CON...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'place': {'type': 'uri', 'value': 'http://ww...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,exception: query is empty,None
2779,"""Find the number of unique awards won by indiv...",,LLMTranslator,,"Unexpected err.msg=""The LLM result doesn't mat...",True,None,"[""Write a query to find the number of occurren...","""Find the number of unique awards won by indiv...",SELECT (COUNT(?item) AS ?count) ?otheraward ?o...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'otheraward': {'type': 'uri', 'value': 'http...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,exception: query is empty,None
1624,"""Find entities in Wikidata that have a linked ...",,LLMTranslator,,"Unexpected err.msg=""The LLM result doesn't mat...",True,None,"[""Find all items on Wikidata that have a linke...","""Find entities in Wikidata that have a linked ...",SELECT ?item ?itemLabel ?3dmodel\nWHERE {\n?it...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,exception: query is empty,None
662,"""Write a SparQL query to retrieve the name, or...",,LLMTranslator,,"Unexpected err.msg=""The LLM result doesn't mat...",True,None,"[""Write a SparQL query to retrieve the name, l...","""Write a SparQL query to retrieve the name, or...",SELECT ?doctor ?doctorLabel ?ordinal ?performe...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'doctor': {'type': 'uri', 'value': 'http://w...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,exception: query is empty,None


In [3]:
print(f"{len(dataset)=}")
print(f"{len(dataset.loc[dataset['has_error']])=}")

len(dataset)=118
len(dataset.loc[dataset['has_error']])=118


In [8]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset


In [10]:
p = "../outputs/batch_run/LM_experiment/execution/CL-7b-Instruct_oawbn8-rv256-rm1-ld0-bs2-ga8-gc1-p0-nta0-e3-ctx3072-qno-template-template-stsparql_engpeft-dgreedy-t02-topp095-cdbf16_executed.parquet.gzip"
dataset = load_dataset(p)

dataset.head()

,row,last_executed_step,to_be_executed_step,translated_prompt,extracted_entities,extracted_properties,linked_entities,linked_properties,output,status,has_error,basic_input,templated_input,target_raw,target_template,gold_execution,gold_executed_query,execution,executed_query
index,,,,,,,,,,,,,,,,,,,
1190,"""Write a SparQL query to retrieve the names, l...",SimplePlaceholderFiller,,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[human, scientist]","[instance of, occupation, date of death, image]","[[human, [Q5, human]], [scientist, [Q901, scie...","[[instance of, [P31, instance of]], [occupatio...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,,False,"[""Write a SparQL query to retrieve the distinc...","""Write a SparQL query to retrieve the names, l...",SELECT DISTINCT ?author ?authorLabel ?image ?d...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'author': {'type': 'uri', 'value': 'http://w...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...",PREFIX xsd: <http://www.w3.org/2001/XMLSchema#...
1927,"""Write a SparQL query to retrieve the start ti...",SimplePlaceholderFiller,,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,[country],"[instance of, image]","[[country, [Q6256, country]]]","[[instance of, [P31, instance of]], [image, [P...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,,False,"[""Write a SparQL query to retrieve the distinc...","""Write a SparQL query to retrieve the start ti...",SELECT DISTINCT ?marriage ?marriageLabel ?time...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'marriage': {'type': 'uri', 'value': 'http:/...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,exception: 400 Client Error: Bad Request for u...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
1160,"""Write a SparQL query to find the name, popula...",SimplePlaceholderFiller,,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[kindergarten, Italy]","[instance of, country, population, coordinate ...","[[kindergarten, [Q126807, kindergarten]], [Ita...","[[instance of, [P31, instance of]], [country, ...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,,False,"[""Write a SparQL query to retrieve the name, l...","""Write a SparQL query to find the name, popula...",SELECT ?city ?cityLabel ?coord ?ratioBin ?rgb\...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'city': {'type': 'uri', 'value': 'http://www...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,[],PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
2514,"""Write a SparQL query to retrieve the coordina...",SimplePlaceholderFiller,,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[tram stop, tram line]","[instance of, coordinate location, connecting ...","[[tram stop, [Q2175765, tram stop]], [tram lin...","[[instance of, [P31, instance of]], [coordinat...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,,False,"[""Write a SparQL query to retrieve the tramway...","""Write a SparQL query to retrieve the coordina...",SELECT ?st ?stLabel ?sl ?coords ?line ?layer ?...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'st': {'type': 'uri', 'value': 'http://www.w...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,[],PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
1773,"""Write a SparQL query to find the latitude and...",SimplePlaceholderFiller,,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,[India],"[country, coordinate location, twin city, coor...","[[India, [Q668, India]]]","[[country, [P17, country]], [coordinate locati...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,,False,"[""Write a SparQL query to retrieve distinct ci...","""Write a SparQL query to find the latitude and...",SELECT DISTINCT ?cds ?line ?item ?itemLabel ?l...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'twin': {'type': 'uri', 'value': 'http://www...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...


In [26]:
entry = dataset.sample()

print(entry.index.item())

print(entry['templated_input'].item())
print()
print(entry['target_template'].item())
print()
print(entry['translated_prompt'].item())

2618
"Write a SparQL query to retrieve the top 100 candidates in the French legislative elections of 2022, along with their labels and the labels of their electoral districts."

PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX schema: <http://schema.org/>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>

SELECT ?item ?itemLabel ?circo ?circoLabel (COUNT(*) AS ?count) WHERE {
?item wdt:[property:instance of] wd:[entity:human];
p:[property:candidacy in election] ?statement.
?statement ps:[property:candidacy in election] wd:[entity:2022 French legislative election].
OPTIONAL { ?statement pq:[property:electoral district] ?circo. }
?sitelink schema:about ?item.
FILTER (SUBSTR(str(?sitelink), 11, 15) = ".wikipedia.org/") .
SERVICE wikibase:label { bd:servi

In [24]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset
import json
import pandas as pd

In [26]:
versions = ['final_fq18_with_limit_10', 'final_fq18_without_limit']
ver_num = 1

split_test = load_dataset(f"../outputs/dataset_pipeline/{versions[ver_num]}/{versions[ver_num]}-split_test.pkl")
split_valid = load_dataset(f"../outputs/dataset_pipeline/{versions[ver_num]}/{versions[ver_num]}-split_valid.pkl")
split_train = load_dataset(f"../outputs/dataset_pipeline/{versions[ver_num]}/{versions[ver_num]}-split_train.pkl")
executed = load_dataset(f"../outputs/dataset_pipeline/{versions[ver_num]}/generated_prompt-executed.parquet.gzip")

In [27]:
print(f"Length of split_test: {len(split_test)}")
print(f"Length of split_valid: {len(split_valid)}")
print(f"Length of split_train: {len(split_train)}")
print(f"Length of executed: {len(executed)}")
print(f"test + valid + train: {len(split_test) + len(split_train) + len(split_valid)}")

Length of split_test: 497
Length of split_valid: 125
Length of split_train: 1863
Length of executed: 2771
test + valid + train: 2485


In [28]:
split_test.head()

,basic_input,templated_input,target_raw,target_template,gold_execution,gold_executed_query
index,,,,,,
122,"[""Generate a bubble chart showing the number o...","[""Write a SparQL query to find the number of u...",SELECT (COUNT(DISTINCT ?mp) as ?mps) ?job ?job...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'job': {'type': 'uri', 'value': 'http://www....",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
2497,"[""Write a SparQL query to retrieve the count a...","[""Write a SparQL query to find the top 10 prop...",SELECT ?property ?propertyLabel ?count ?report...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'property': {'type': 'uri', 'value': 'http:/...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
2207,"[""Write a SparQL query to retrieve the name, l...","[""Write a SparQL query to retrieve the names, ...",SELECT ?street ?streetLabel ?layer ?coords\nWH...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'street': {'type': 'uri', 'value': 'http://w...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
1085,"[""Find and retrieve the names, descriptions, a...","[""Write a SparQL query to retrieve the labels ...",SELECT ?item ?itemLabel ?itemDescription ?valu...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
2323,"[""Write a SparQL query to retrieve the journal...","[""Write a SparQL query to retrieve the start t...",SELECT ?journal ?journalLabel ?started\nWHERE ...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'journal': {'type': 'uri', 'value': 'http://...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...


In [30]:
number_of_duplicates = lambda x, y: x.duplicated(subset=[y]).sum()
print(f"Number of duplicates")
print(f"test: {number_of_duplicates(split_test, 'target_raw')}")
print(f"valid: {number_of_duplicates(split_valid, 'target_raw')}")
print(f"train: {number_of_duplicates(split_train, 'target_raw')}")
print(f"executed: {number_of_duplicates(executed, 'query')}")

Number of duplicates
test: 0
valid: 0
train: 0
executed: 0
